In [2]:
import os
import csv
import torch
import librosa
from datasets import Dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import evaluate
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import TrainingArguments, Trainer
from P2_Transformers_Speech_Classification.project_2 import predict_single_audio, create_dataset

In [3]:
os.getcwd()

'C:\\Users\\wojew\\Documents\\DS\\sem1\\Deep Learning\\CINIC10_Proj1\\P2_Transformers_Speech_Classification'

# Load model

In [4]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

## Predict single audio

In [3]:
predict_single_audio(model, processor, 'data/train/audio/happy/0ab3b47d_nohash_0.wav')

Transcription for data/train/audio/happy/0ab3b47d_nohash_0.wav: HAPPY


'HAPPY'

# Create dataset

In [8]:
audio_dir = "data/train/audio/" 
dataset = create_dataset(audio_dir, processor, 'testing_list')
dataset

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 6835
})

In [19]:
dataset[0]

{'input_values': [0.019689498469233513,
  0.025986898690462112,
  0.052525945007801056,
  0.08311332017183304,
  0.07760309427976608,
  0.09773228317499161,
  0.11763656884431839,
  0.11437541246414185,
  0.11133916676044464,
  0.10054362565279007,
  0.0420677624642849,
  -0.012922042049467564,
  -0.03856145963072777,
  -0.08624178171157837,
  -0.12571299076080322,
  -0.10951967537403107,
  -0.06408842653036118,
  -0.03316368907690048,
  0.01271737553179264,
  0.045891184359788895,
  0.07366721332073212,
  0.10166815668344498,
  0.11122671514749527,
  0.09244696795940399,
  0.06275922060012817,
  0.05533728376030922,
  0.015303808264434338,
  -0.016857918351888657,
  -0.013484310358762741,
  -0.016070742160081863,
  -0.010897877626121044,
  0.015191354788839817,
  0.02126384899020195,
  0.030372587963938713,
  0.060510147362947464,
  0.05702408775687218,
  0.05601200833916664,
  0.0691690742969513,
  0.04859007149934769,
  0.007319603580981493,
  -0.012134866788983345,
  -0.01550847478

# Fine-tune model

In [17]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments

model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

def prepare_dataset(example):
    input_values = processor(example["input_values"], return_tensors="pt").input_values
    labels = processor(example["labels"], return_tensors="pt", padding='longest').input_values
    return {"input_values": input_values, "labels": labels}

encoded_dataset = dataset.map(prepare_dataset)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Map:   0%|          | 0/6835 [00:00<?, ? examples/s]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_ra

In [18]:
encoded_dataset[0]

{'input_values': [[0.019689515233039856,
   0.025986919179558754,
   0.05252597853541374,
   0.08311337977647781,
   0.07760314643383026,
   0.09773235023021698,
   0.11763664335012436,
   0.11437548696994781,
   0.11133924126625061,
   0.10054369270801544,
   0.042067792266607285,
   -0.012922044843435287,
   -0.03856147825717926,
   -0.08624182641506195,
   -0.1257130652666092,
   -0.10951973497867584,
   -0.06408845633268356,
   -0.033163703978061676,
   0.01271738763898611,
   0.04589121416211128,
   0.0736672654747963,
   0.10166822373867035,
   0.11122678965330124,
   0.09244702756404877,
   0.06275926530361176,
   0.055337321013212204,
   0.015303822234272957,
   -0.016857922077178955,
   -0.01348431408405304,
   -0.016070745885372162,
   -0.010897879488766193,
   0.015191368758678436,
   0.021263867616653442,
   0.030372612178325653,
   0.06051018834114075,
   0.05702412500977516,
   0.056012045592069626,
   0.06916912645101547,
   0.04859010502696037,
   0.007319612428545952,


In [7]:
training_args = TrainingArguments(
    output_dir="./output_dir",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor,
)

trainer.train()

C:\Users\wojew\Documents\DS\venv\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


ValueError: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.